# Demo

## Download data from Kaggle

### Install Kaggle python API

In [1]:
! pip install kaggle

### Authenticating with Kaggle 

In [2]:
! kaggle datasets download "ranjeetshrivastav/fraud-detection-dataset"

fraud-detection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
! unzip -o fraud-detection-dataset.zip

Archive:  fraud-detection-dataset.zip
  inflating: transactions.gz         
  inflating: transactions/transactions.txt  


In [4]:
import os
import numpy as np
import pandas as pd
import networkx as nx
from nodevectors import Node2Vec
import xgboost as xgb
from fucc.inductive_step import inductive_pooling
from fucc.metrics import plot_ap, get_optimal_f1_cutoff, get_confusion_matrix
from sklearn.metrics import average_precision_score
import logging
logging.basicConfig(level=logging.INFO)

Dataset source: https://www.kaggle.com/ranjeetshrivastav/fraud-detection-dataset

In [5]:
# Parameters
dimensions = 32
walk_len = 80
walk_num = 10
window_size = 5
workers = 8

## Load Data

In [6]:
df = pd.read_json('/Users/raf/Dropbox/DOC/data/fraud_datasets/archive/transactions/transactions.txt',  lines=True, convert_dates=[4])

In [7]:
df.iloc[:, 4] = pd.to_datetime(df.iloc[:, 4])

In [8]:
df = df.sort_values('transactionDateTime')
df.loc[:, 'TX_ID'] = range(df.shape[0])

In [9]:
df = df.rename(columns={"merchantName":"TERM_MIDUID", "customerId":"CARD_PAN_ID", "isFraud": "TX_FRAUD" })

In [10]:
df_train = df.iloc[:400000]
df_test = df.iloc[400000:500000]

## Create network

In [11]:
G = nx.Graph()
G.add_nodes_from(df_train.TERM_MIDUID.unique(), type='merchant')
G.add_nodes_from(df_train.CARD_PAN_ID.unique(), type='cardholder')
G.add_nodes_from(df_train.TX_ID.unique(), type='transaction')

G.add_edges_from(zip(df_train.CARD_PAN_ID, df_train.TX_ID))
G.add_edges_from(zip(df_train.TX_ID, df_train.TERM_MIDUID))

In [12]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 407337
Number of edges: 800000
Average degree:   3.9280


## Deepwalk

In [ ]:
# Fit embedding model to graph
g2v = Node2Vec(
    n_components=dimensions,
    walklen = walk_len,
    epochs = walk_num,
    w2vparams={'workers': workers, 'window': window_size}
)

g2v.fit(G)
model = g2v.model

Making walks... Done, T=83.13
Mapping Walk Names... 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/Users/raf/opt/anaconda3/envs/RLFRaud/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-173df08cb0e2>", line 9, in <module>
    g2v.fit(G)
  File "/Users/raf/opt/anaconda3/envs/RLFRaud/lib/python3.6/site-packages/nodevectors/node2vec.py", line 117, in fit
    self.walks[col] = self.walks[col].map(node_dict).astype(str)
  File "/Users/raf/opt/anaconda3/envs/RLFRaud/lib/python3.6/site-packages/pandas/core/frame.py", line 3044, in __setitem__
    self._set_item(key, value)
  File "/Users/raf/opt/anaconda3/envs/RLFRaud/lib/python3.6/site-packages/pandas/core/frame.py", line 3121, in _set_item
    NDFrame._set_item(self, key, value)
  File "/Users/raf/opt/anaconda3/envs/RLFRaud/lib/python3.6/site-packages/pandas/core/generic.py", line 3582, in _set_item
    NDFrame._iset_item(self, loc, value)
  File "/Users/raf/opt/anaconda3/envs/RLF

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/Users/raf/opt/anaconda3/envs/RLFRaud/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-173df08cb0e2>", line 9, in <module>
    g2v.fit(G)
  File "/Users/raf/opt/anaconda3/envs/RLFRaud/lib/python3.6/site-packages/nodevectors/node2vec.py", line 117, in fit
    self.walks[col] = self.walks[col].map(node_dict).astype(str)
  File "/Users/raf/opt/anaconda3/envs/RLFRaud/lib/python3.6/site-packages/pandas/core/frame.py", line 3044, in __setitem__
    self._set_item(key, value)
  File "/Users/raf/opt/anaconda3/envs/RLFRaud/lib/python3.6/site-packages/pandas/core/frame.py", line 3121, in _set_item
    NDFrame._set_item(self, key, value)
  File "/Users/raf/opt/anaconda3/envs/RLFRaud/lib/python3.6/site-packages/pandas/core/generic.py", line 3582, in _set_item
    NDFrame._iset_item(self, loc, value)
  File "/Users/raf/opt/anaconda3/envs/RLF

In [13]:
embeddings = {}
for i in df_train.TX_ID:
    embeddings[i] = model.wv[str(i)]


embeddings = pd.DataFrame().from_dict(embeddings, orient='index')

ERROR! Session/line number was not unique in database. History logging moved to new session 4473


NameError: name 'model' is not defined

In [ ]:
df_train = df_train.merge(embeddings, left_on='TX_ID', right_index=True)

In [ ]:
df_train.head()

## Inductive Pooling

In [ ]:
results = inductive_pooling(df=df_test, embeddings=embeddings, G=G, workers=workers)

In [ ]:
df_new_embeddings = pd.concat([pd.DataFrame(li).transpose() for li in results])

In [ ]:
df_new_embeddings.index = df_test.TX_ID
df_test = df_test.merge(df_new_embeddings, left_on='TX_ID', right_index=True)

## XGBoost Classifier

In [ ]:
embedding_features = [i for i in range(dimensions)]

In [ ]:
X_train = df_train[embedding_features].iloc[:int(df_train.shape[0]*0.8)]
X_val = df_train[embedding_features].iloc[int(df_train.shape[0]*0.8):]
y_train = df_train.TX_FRAUD.iloc[:int(df_train.shape[0]*0.8)]
y_val = df_train.TX_FRAUD.iloc[int(df_train.shape[0]*0.8):]

X_test = df_test[embedding_features]
y_test = df_test.TX_FRAUD

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
xgb_params = {
    'eval_metric': ['auc','aucpr', 'logloss'],
    'objective':'binary:logistic',
    'n_estimators': 300,
    'n_jobs':8,
    'learning_rate':0.1,
    'seed':42,
    'colsample_bytree':0.6,
    'colsample_bylevel':0.9,
    'subsample':0.9
}

In [ ]:
model = xgb.train(xgb_params, dtrain, num_boost_round=xgb_params['n_estimators'], evals=[(dval, 'val'), (dtrain, 'train')], early_stopping_rounds=int(xgb_params['n_estimators']/2))

In [ ]:
y_pred_proba = model.predict(dtest)

## Evaluation

In [ ]:
ap = average_precision_score(y_test, y_pred_proba)
print("Average Precision: ", np.round(ap,2))

In [ ]:
fig = plot_ap(y_test, y_pred_proba)

In [ ]:
optimal_threshold, optimal_f1_score = get_optimal_f1_cutoff(y_test, y_pred_proba)
print("F1 Score: ", np.round(optimal_f1_score, 4))

In [ ]:
cm = get_confusion_matrix(y_test, y_pred_proba, optimal_threshold)
print("Confusion Matrix: \n", cm)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_test, y_pred_proba)